# Databricks RAGアプリチュートリアル - 2. RAGエージェントの構築

このノートブックでは、前回のワークショップで作成したVector Search Indexを活用して、より高度なTool-calling RAG（Retrieval-Augmented Generation）エージェントを構築します。

## このノートブックで学習する内容
1. **Mosaic AI Agent Framework**の詳細理解
2. **LangGraph**を使用した高度なエージェント実装
3. **Tool-calling機能**によるマルチステップ推論
4. **MLflow ChatAgent**インターフェースの活用
5. **Unity Catalog統合**による企業レベルのガバナンス
6. **デプロイとモニタリング**のベストプラクティス

## 実行環境
このノートブックはサーバーレスコンピュートでの実行を想定しています。

## 前提条件
- 1_workshop_vector_search.ipynbでVector Search Indexが作成済みであること
- 必要なエンドポイントへのアクセス権限があること

---

## LangGraphとTool-calling Agentについて

### LangGraphとは
LangGraphは、LangChainエコシステムの一部として開発された、複雑なマルチエージェントワークフローを構築するためのライブラリです。

**主な特徴:**
- **State Management**: 複雑な状態管理
- **Conditional Logic**: 条件分岐による動的フロー制御
- **Tool Integration**: 外部ツールとのシームレスな統合
- **Human-in-the-loop**: 人間の介入ポイントの設定

### Tool-calling Agentの利点
1. **マルチステップ推論**: 複数のツールを組み合わせた複雑なタスクの実行
2. **動的ツール選択**: 文脈に応じた最適なツールの自動選択
3. **エラー処理**: ツール実行失敗時の自動回復
4. **拡張性**: 新しいツールの簡単な追加

In [0]:
# 必要なライブラリのインストール
# MLflow 3.0以降とLangGraphを使用した高度なエージェント開発環境をセットアップ
%pip install -U -qqqq mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents uv
dbutils.library.restartPython()

## 3-1. エージェントの実装

### Mosaic AI Agent Frameworkの詳細

Mosaic AI Agent Frameworkは、Enterprise-gradeのRAGアプリケーション開発のために設計された包括的なソリューションです。

**フレームワークの核心要素:**
1. **ChatAgent Interface**: 標準化されたマルチターン会話インターフェース
2. **Tool Integration**: 外部システムとの統合機能
3. **State Management**: 会話状態の永続化
4. **Security & Governance**: Unity Catalogによるアクセス制御

### LangGraphによるワークフロー設計

以下でエージェントコードを単一のセルで定義し、`%%writefile`マジックコマンドを使用してローカルPythonファイルに出力します。これにより、後続のロギングとデプロイメントが容易になります。

**実装するエージェントアーキテクチャ:**
- **Entry Point**: ユーザーからの質問受付
- **Tool Selection**: 適切なツールの動的選択
- **Conditional Edges**: 結果に基づく分岐処理
- **Response Generation**: 最終回答の生成

より多くのツールの例については、[公式ドキュメント](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/agent-tool)をご参照ください。

In [0]:
%%writefile agent.py
"""
Databricks Tool-calling RAG Agent Implementation

このファイルは、LangGraphとMosaic AI Agent Frameworkを使用した
高度なTool-calling RAGエージェントの実装です。

主な機能:
- Vector Searchを活用した文書検索
- Unity Catalog Functionとの統合
- マルチターン会話の管理
- ストリーミング対応
"""

from typing import Any, Generator, Optional, Sequence, Union

import mlflow
from databricks_langchain import (
    ChatDatabricks,
    VectorSearchRetrieverTool,
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)

# MLflowの自動ログ機能を有効化
mlflow.langchain.autolog()

# Unity Catalog Function用のクライアント設定
client = DatabricksFunctionClient()
set_uc_function_client(client)

############################################
# LLMエンドポイントとシステムプロンプトの定義
############################################
LLM_ENDPOINT_NAME = "databricks-claude-sonnet-4"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

# システムプロンプト: エージェントの基本的な動作を定義
system_prompt = """あなたは生成AI開発に関する専門的なアシスタントです。
登録されたVector Search Indexを活用して、
ユーザーの質問に対して正確で有用な回答を提供してください。

以下のガイドラインに従ってください：
1. 利用可能なツールを効果的に活用してください
2. 検索された文書の内容に基づいて回答してください  
3. 不確かな情報については推測を避けてください
4. 具体的で実用的なアドバイスを提供してください
5. 日本語で丁寧に回答してください

必要に応じて複数のツールを組み合わせて使用し、
ユーザーにとって最も有益な情報を提供してください。"""

###############################################################################
## エージェントツールの定義
## テキスト生成以外のデータ取得やアクション実行を可能にするツール群
## 詳細な例については以下のドキュメントを参照:
## https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/agent-tool
###############################################################################
tools = []

# Unity CatalogのUDFをエージェントツールとして使用
# 必要に応じて関数名を追加してください
uc_tool_names = []
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools.extend(uc_toolkit.tools)

# Vector Search Indexごとにretrieverツールを作成
# 詳細: https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/unstructured-retrieval-tools
vector_search_index_tools = [
    VectorSearchRetrieverTool(
        index_name="skato.rag_workshop.chunked_document_vs_index",
        tool_description="生成AI開発に関する技術文書やベストプラクティスを検索するためのツールです。GenAI開発ワークフロー、MLOps、エージェント設計などについて質問がある場合に使用してください。"
    )
]
tools.extend(vector_search_index_tools)

#####################
## エージェントロジックの定義
#####################

def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Union[Sequence[BaseTool], ToolNode],
    system_prompt: Optional[str] = None,
) -> CompiledGraph:
    """
    Tool-calling機能を持つエージェントを作成
    
    Args:
        model: 使用するLLMモデル
        tools: エージェントが使用可能なツールのリスト
        system_prompt: システムプロンプト
    
    Returns:
        コンパイル済みのLangGraphワークフロー
    """
    # モデルにツールをバインド
    model = model.bind_tools(tools)

    # 次のノードを決定する関数
    def should_continue(state: ChatAgentState):
        """
        エージェントがツールを呼び出すかどうかを判定
        """
        messages = state["messages"]
        last_message = messages[-1]
        # Function callがある場合は継続、そうでなければ終了
        if last_message.get("tool_calls"):
            return "continue"
        else:
            return "end"

    # システムプロンプトの前処理
    if system_prompt:
        preprocessor = RunnableLambda(
            lambda state: [{"role": "system", "content": system_prompt}]
            + state["messages"]
        )
    else:
        preprocessor = RunnableLambda(lambda state: state["messages"])
    
    model_runnable = preprocessor | model

    def call_model(
        state: ChatAgentState,
        config: RunnableConfig,
    ):
        """
        LLMモデルを呼び出し、レスポンスを処理
        """
        response = model_runnable.invoke(state, config)
        return {"messages": [response]}

    # LangGraphワークフローの構築
    workflow = StateGraph(ChatAgentState)

    # ノードの追加
    workflow.add_node("agent", RunnableLambda(call_model))
    workflow.add_node("tools", ChatAgentToolNode(tools))

    # エントリーポイントの設定
    workflow.set_entry_point("agent")
    
    # 条件分岐エッジの追加
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "continue": "tools",  # ツール実行へ
            "end": END,          # 処理終了
        },
    )
    
    # ツールからエージェントへのエッジ
    workflow.add_edge("tools", "agent")

    return workflow.compile()


class LangGraphChatAgent(ChatAgent):
    """
    MLflow ChatAgentインターフェースを実装したLangGraphエージェント
    
    マルチターン会話とストリーミング対応を提供
    """
    
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        """
        同期的な予測処理
        
        Args:
            messages: 会話履歴
            context: 追加のコンテキスト情報
            custom_inputs: カスタム入力パラメータ
            
        Returns:
            ChatAgentResponse: エージェントの応答
        """
        request = {"messages": self._convert_messages_to_dict(messages)}

        messages = []
        # ワークフローを実行し、すべての更新を収集
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                messages.extend(
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                )
        return ChatAgentResponse(messages=messages)

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        """
        ストリーミング予測処理
        
        Args:
            messages: 会話履歴
            context: 追加のコンテキスト情報
            custom_inputs: カスタム入力パラメータ
            
        Yields:
            ChatAgentChunk: エージェントの応答チャンク
        """
        request = {"messages": self._convert_messages_to_dict(messages)}
        # ワークフローをストリーミング実行
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg}) for msg in node_data["messages"]
                )


# エージェントオブジェクトの作成
# MLflow推論時に使用するエージェントとして設定
agent = create_tool_calling_agent(llm, tools, system_prompt)
AGENT = LangGraphChatAgent(agent)
mlflow.models.set_model(AGENT)

## 3-2. エージェントのテスト

### MLflow Tracingについて

このノートブックでは`mlflow.langchain.autolog()`を呼び出しているため、エージェントが実行する各ステップのトレースをMLflow UIで確認することができます。

**Tracingの利点:**
1. **デバッグ支援**: 各ツール呼び出しの詳細な追跡
2. **パフォーマンス分析**: レスポンス時間とボトルネックの特定
3. **品質監視**: 入力・出力の品質管理
4. **運用監視**: プロダクション環境でのエージェント動作監視

### インタラクティブテスト

エージェントとの対話を通じて出力をテストします。以下のプレースホルダー入力を、あなたのエージェントに適した具体的な例に置き換えてください。

**テスト観点:**
- Vector Searchによる関連文書の検索精度
- LLMによる回答生成の品質
- Tool-calling機能の適切な動作
- マルチステップ推論の実行

In [0]:
# Pythonライブラリの再起動
# agent.pyファイルの変更を反映するため
dbutils.library.restartPython()

In [0]:
# エージェントのインポートとBasic Test
from agent import AGENT

# 基本的な動作テスト
print("=== 基本動作テスト ===")
response = AGENT.predict({"messages": [{"role": "user", "content": "Hello!"}]})
print("エージェントからの応答:")
for message in response.messages:
    print(f"Role: {message.role}")
    print(f"Content: {message.content}")
    print("-" * 50)

In [0]:
# ストリーミング機能のテスト
print("=== ストリーミングテスト ===")
print("質問: 生成AIの開発ワークフローについて教えてください")
print("\n--- ストリーミング応答 ---")

for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "生成AIの開発ワークフローについて教えてください"}]}
):
    if hasattr(event, 'delta') and event.delta:
        if hasattr(event.delta, 'content') and event.delta.content:
            print(event.delta.content, end="", flush=True)
        elif hasattr(event.delta, 'tool_calls') and event.delta.tool_calls:
            print(f"\n[ツール呼び出し: {event.delta.tool_calls}]")
    print() # 改行

## 3-3. エージェントのMLflowモデルとしてのログ記録

### リソース指定とセキュリティ

デプロイメント時の自動認証パススルーのために必要なDatabricksリソースを特定します。

**重要な考慮事項:**
- **Vector Search Index**: Unity Catalog Functionがvector search indexにアクセスする場合、依存するインデックスをリソースとして含める必要があります
- **External Functions**: 外部関数を活用する場合、UC接続オブジェクトをリソースとして含める必要があります
- **自動認証**: 詳細については[公式ドキュメント](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#specify-resources-for-automatic-authentication-passthrough)を参照

### Models from Code

`agent.py`ファイルからエージェントをコードとしてログします。詳細は[MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code)を参照してください。

**Benefits:**
1. **バージョン管理**: コードの変更履歴の追跡
2. **再現性**: 同じ環境での確実な再現
3. **デプロイメント**: プロダクション環境への安全なデプロイ
4. **依存関係管理**: 必要なライブラリの自動管理

In [0]:
# デプロイメント時の自動認証パススルー用のDatabricksリソース特定
import mlflow
from agent import LLM_ENDPOINT_NAME, tools
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

print("=== リソース設定 ===")

# LLMエンドポイントをリソースとして追加
resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
print(f"LLMエンドポイント: {LLM_ENDPOINT_NAME}")

# 各ツールの依存リソースを確認・追加
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        print(f"Vector Search Tool: {tool.index_name}")
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        print(f"Unity Catalog Tool: {tool.uc_function_name}")
        # TODO: UC functionが外部接続やvector searchに依存する場合は、
        # それらのリソースを手動で追加する必要があります
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

print(f"総リソース数: {len(resources)}")

# モデルの入力例を定義
input_example = {
    "messages": [
        {
            "role": "user",
            "content": "LLMエージェントとは何ですか？"
        }
    ]
}

print("\n=== MLflowモデルログ記録 ===")

# MLflow実行の開始とモデルログ
with mlflow.start_run():
    # パフォーマンス情報の記録
    mlflow.log_param("llm_endpoint", LLM_ENDPOINT_NAME)
    mlflow.log_param("num_tools", len(tools))
    mlflow.log_param("framework", "LangGraph")
    
    # モデルのログ記録
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}",
            f"mlflow=={get_distribution('mlflow').version}",
            f"databricks-langchain=={get_distribution('databricks-langchain').version}",
            f"langgraph=={get_distribution('langgraph').version}",
        ],
    )
    
    print(f"モデルがログされました: {logged_agent_info.model_uri}")
    print(f"Run ID: {logged_agent_info.run_id}")

## 3-4. Agent Evaluationによるエージェント評価

### MLflow 3.0 Agent Evaluationについて

[Agent Evaluation](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/)は、生成AIエージェントの品質を定量的に評価するための強力なフレームワークです。

**主な特徴:**
- **LLM-as-a-Judge**: 他のLLMモデルを評価者として活用
- **事前定義メトリクス**: 標準的な品質指標
- **カスタムメトリクス**: ドメイン特有の評価基準
- **継続的監視**: プロダクション環境でのリアルタイム監視

### 評価のワークフロー

1. **評価データセット作成**: 期待される応答と共にテストケースを定義
2. **メトリクス選択**: 適切な評価指標の選択
3. **評価実行**: エージェントのパフォーマンス測定
4. **結果分析**: MLflowでの品質メトリクス追跡

評価データセットの質問や期待される応答を編集し、エージェントの改善に合わせて評価を実行できます。MLflowを活用して計算された品質メトリクスを追跡することが可能です。

[事前定義されたLLMスコアラー](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/predefined-judge-scorers)でエージェントを評価するか、[カスタムメトリクス](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/custom-scorers)の追加をお試しください。

In [0]:
# MLflow Agent Evaluationの実行
import mlflow
from mlflow.genai.scorers import RelevanceToQuery, Safety, RetrievalRelevance, RetrievalGroundedness

print("=== Agent Evaluation実行 ===")

# 評価データセットの定義
# 実際の使用ケースに応じてデータセットを拡張してください
eval_dataset = [
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "生成AIの開発ワークフローについて詳しく教えてください"
                }
            ]
        },
        "expected_response": None  # 期待される応答がある場合はここに記載
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user", 
                    "content": "MLOpsのベストプラクティスは何ですか？"
                }
            ]
        },
        "expected_response": None
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "Vector SearchとRAGの関係について説明してください"
                }
            ]
        },
        "expected_response": None
    }
]

print(f"評価データセット: {len(eval_dataset)} 件のテストケース")

# エージェント評価の実行
# 使用可能なスコアラー:
# - RelevanceToQuery: 質問に対する回答の関連性
# - Safety: 安全性（有害性の検出）
# - RetrievalRelevance: 検索された文書の関連性
# - RetrievalGroundedness: 文書に基づいた回答かどうか
eval_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=lambda messages: AGENT.predict({"messages": messages}),
    scorers=[
        RelevanceToQuery(),  # 質問関連性
        Safety(),           # 安全性
        # 以下は必要に応じて追加
        # RetrievalRelevance(),   # 検索関連性
        # RetrievalGroundedness() # 文書根拠性
    ],
)

print("評価が完了しました！")
print("MLflow UIで詳細な評価結果を確認してください")

## 3-5. エージェントのデプロイ前検証

### デプロイ前検証の重要性

エージェントを登録・デプロイする前に、[mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) APIを通じて事前検証を実行します。

**検証の目的:**
1. **環境一貫性**: 開発環境とプロダクション環境の動作確認
2. **依存関係チェック**: 必要なライブラリとリソースの確認
3. **入力形式検証**: 期待される入力形式の確認
4. **エラー処理**: 例外処理の動作確認

**検証プロセス:**
- ローカル環境でのモデル復元
- テスト入力による動作確認
- レスポンス形式の検証
- パフォーマンス測定

詳細については[公式ドキュメント](https://learn.microsoft.com/azure/databricks/machine-learning/model-serving/model-serving-debug#validate-inputs)を参照してください。

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data={"messages": [{"role": "user", "content": "Hello!"}]},
    env_manager="virtualenv",
)

# デプロイ前検証の実行
# print("=== デプロイ前検証開始 ===")

# try:
#     # テスト用入力データ
#     test_input = {
#         "messages": [
#             {
#                 "role": "user", 
#                 "content": "生成AIエージェントの基本的な仕組みについて教えてください"
#             }
#         ]
#     }
    
#     print("テスト入力:", test_input["messages"][0]["content"])
#     print("モデル復元中...")
    
#     # MLflowモデルの予測実行
#     # env_manager="uv"を使用して依存関係を管理
#     result = mlflow.models.predict(
#         model_uri=f"runs:/{logged_agent_info.run_id}/agent",
#         input_data=test_input,
#         env_manager="uv",
#     )
    
#     print("✅ 検証成功!")
#     print("\n--- 応答結果 ---")
#     if hasattr(result, 'messages'):
#         for message in result.messages:
#             print(f"Role: {message.role}")
#             print(f"Content: {message.content[:200]}...")
#             print("-" * 30)
#     else:
#         print("Result:", str(result)[:200])
    
# except Exception as e:
#     print(f"❌ 検証失敗: {str(e)}")
#     print("デプロイ前に問題を修正する必要があります")

# print("\n=== 検証完了 ===")

## 3-6. Unity Catalogへのモデル登録

### Unity Catalogによるガバナンス

Unity Catalogは、Databricksの統一ガバナンスソリューションです。MLflowモデルをUnity Catalogに登録することで、以下の利点が得られます。

**主な利点:**
1. **アクセス制御**: 細かい権限管理とセキュリティ
2. **監査ログ**: 完全なアクセス履歴の追跡
3. **メタデータ管理**: モデルの詳細情報とリネージ
4. **バージョン管理**: モデルの変更履歴とロールバック機能
5. **組織全体での共有**: チーム間でのモデル共有

### 登録手順

以下の`catalog`、`schema`、`model_name`を更新して、MLflowモデルをUnity Catalogに登録してください。

**命名規則のベストプラクティス:**
- **catalog**: 組織またはビジネスユニット名
- **schema**: プロジェクトまたはドメイン名  
- **model_name**: 機能を表す分かりやすい名前

In [0]:
# Unity Catalogへのモデル登録
print("=== Unity Catalog登録 ===")

# Unity CatalogをMLflowレジストリとして設定
mlflow.set_registry_uri("databricks-uc")

# Unity Catalogモデルの定義
# 実際の環境に合わせて更新してください
catalog = "skato"  # あなたのカタログ名
schema = "rag_workshop"  # スキーマ名
model_name = "tool_calling_rag_agent"  # モデル名

UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

print(f"登録先: {UC_MODEL_NAME}")

try:
    # Unity Catalogにモデルを登録
    uc_registered_model_info = mlflow.register_model(
        model_uri=logged_agent_info.model_uri, 
        name=UC_MODEL_NAME
    )
    
    print("✅ Unity Catalogへの登録が完了しました!")
    print(f"モデル名: {uc_registered_model_info.name}")
    print(f"バージョン: {uc_registered_model_info.version}")
    print(f"ステータス: {uc_registered_model_info.current_stage}")
    
    # メタデータの設定
    mlflow.set_model_version_tag(
        name=UC_MODEL_NAME,
        version=uc_registered_model_info.version,
        key="framework",
        value="LangGraph"
    )
    
    mlflow.set_model_version_tag(
        name=UC_MODEL_NAME,
        version=uc_registered_model_info.version,
        key="use_case",
        value="RAG Tool-calling Agent"
    )
    
    print("メタデータタグが設定されました")
    
except Exception as e:
    print(f"❌ 登録エラー: {str(e)}")
    print("カタログ、スキーマ、モデル名を確認してください")

## 3-7. エージェントのデプロイメント

### Model Servingについて

Databricks Model Servingは、機械学習モデルをRESTful APIとして本番環境にデプロイするためのマネージドサービスです。

**主な特徴:**
1. **オートスケーリング**: トラフィックに応じた自動スケーリング
2. **A/Bテスト**: 複数のモデルバージョンのトラフィック分割
3. **モニタリング**: リアルタイムのパフォーマンス監視
4. **セキュリティ**: 企業レベルのセキュリティとアクセス制御
5. **高可用性**: フォルトトレラントなインフラストラクチャ

### デプロイメントプロセス

エージェントをサービングエンドポイントとしてデプロイし、REST APIを通じてアクセス可能にします。

In [0]:
# エージェントのデプロイメント
print("=== エージェントデプロイメント ===")

from databricks import agents

try:
    # エージェントをModel Servingエンドポイントとしてデプロイ
    deployment_info = agents.deploy(
        UC_MODEL_NAME, 
        uc_registered_model_info.version, 
        tags={
            "endpointSource": "workshop",
            "framework": "LangGraph",
            "version": "v1.0"
        }
    )
    
    print("✅ デプロイメントが開始されました!")
    print(f"エンドポイント名: {UC_MODEL_NAME.replace('.', '_')}")
    print(f"モデルバージョン: {uc_registered_model_info.version}")
    
    print("\n--- デプロイメント情報 ---")
    print("デプロイメントの完了まで数分かかる場合があります")
    print("以下でエンドポイントの状態を確認できます:")
    print("1. Databricks UIのModel Servingセクション")
    print("2. AI Playground")
    print("3. REST API経由でのアクセス")
    
    print("\n--- 次のステップ ---")
    print("1. エンドポイントの準備完了を待つ")
    print("2. AI Playgroundでテスト")
    print("3. 本番アプリケーションとの統合")
    
except Exception as e:
    print(f"❌ デプロイメントエラー: {str(e)}")
    print("Unity Catalogモデルの登録状況を確認してください")

## まとめと次のステップ

### ワークショップの完了

🎉 **おめでとうございます！** Tool-calling RAGエージェントの開発・評価・デプロイが完了しました。

### このノートブックで学習した内容

1. **LangGraphによる高度なエージェント設計**
   - 複雑なワークフローの実装
   - 条件分岐とツール選択
   - 状態管理とマルチステップ推論

2. **Mosaic AI Agent Frameworkの活用**
   - Unity Catalogとの統合
   - セキュリティとガバナンス
   - Enterprise-gradeの機能

3. **MLflow 3.0による運用管理**
   - モデルのログとバージョン管理
   - 自動評価とモニタリング
   - トレーシングとデバッグ

4. **プロダクションデプロイメント**
   - Model Servingによるスケーラブルなデプロイ
   - REST API経由でのアクセス
   - 継続的な運用監視

### 次のステップ

エージェントがデプロイされた後、以下の活動を通じてさらなる改善を図ることができます：

#### 1. 品質向上
- **AI Playground**でのインタラクティブテスト
- 社内SME（Subject Matter Expert）からのフィードバック収集
- 評価メトリクスの継続的監視

#### 2. 本番統合
- プロダクションアプリケーションへの組み込み
- REST API経由での統合開発
- ユーザーフィードバックの収集

#### 3. スケーリング
- **A/Bテスト**による機能改善
- **多言語対応**の実装
- **カスタムツール**の追加開発

#### 4. 運用最適化
- **パフォーマンス監視**の設定
- **コスト最適化**の実施
- **セキュリティ強化**の継続

### 参考リンク

- [Databricks エージェントデプロイメントガイド](https://learn.microsoft.com/azure/databricks/generative-ai/deploy-agent)
- [AI Playground使用方法](https://learn.microsoft.com/azure/databricks/generative-ai/ai-playground)
- [Model Serving運用ガイド](https://learn.microsoft.com/azure/databricks/machine-learning/model-serving/)
- [Unity Catalogベストプラクティス](https://learn.microsoft.com/azure/databricks/data-governance/unity-catalog/)

---

**これでDatabricks RAGアプリケーション開発のワークショップシリーズが完了です。**
**学習した技術を活用して、組織の生成AIプロジェクトを成功に導いてください！**